# EDA Analysis

In [2]:
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.preprocessing import PowerTransformer
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope


In [3]:
trans = pd.read_csv("transactions_dataset.csv", sep=';')

In [4]:
trans.head()

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051


In [5]:
trans_sample = trans.copy()

In [6]:
covid = pd.to_datetime("2020-03-12")
trans_sample["date_order"] = pd.to_datetime(trans_sample["date_order"])

# time-related features
trans_sample["week"] = trans_sample["date_order"].dt.week
trans_sample["month"] = trans_sample["date_order"].dt.month
#trans_sample["year_month"] = trans_sample["date_order"].dt.strftime("%y%m") #it is to deal with date-time differences
#trans_sample["day"] = trans_sample["date_order"].dt.day
#trans_sample["weekday"] = trans_sample["date_order"].dt.weekday
#trans_sample["weekend"] = trans_sample["weekday"].map(lambda x: 1*(x>4))

C:\Users\Chaouki\AppData\Local\Temp/ipykernel_10300/2789854860.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  trans_sample["week"] = trans_sample["date_order"].dt.week


In [7]:
trans_sample.head()

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,week,month
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732,39,9
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279,39,9
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184,39,9
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200,39,9
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051,39,9


## How many clients per month

In [8]:
analysis0 = pd.pivot_table(
data = trans_sample,
values = "client_id",
index = "month",
aggfunc = "count"
)

In [9]:
analysis0.sort_values(by='client_id', ascending = False) #consistent number of clients throught the year 

,client_id
month,
8,6349231
10,5950650
7,5883583
3,5801338
11,5577607
6,5370479
12,5289941
1,5200638
2,4863553


## How many clients per week

In [10]:
analysis0_1 = pd.pivot_table(
data = trans_sample,
values = "client_id",
index = "week",
aggfunc = "count"
)

In [11]:
analysis0_1.sort_values(by = 'client_id', ascending = False) #there are weeks with higher performance at the end of the year compared to at the beginning of the year 

,client_id
week,
32,1477202
33,1453241
34,1434982
35,1418961
27,1405231
26,1387271
12,1371898
10,1359960
13,1355971


## How many clients per month and week

In [12]:
analysis0_2 = pd.pivot_table(
data = trans_sample,
values = "client_id",
index = ["month","week"],
aggfunc = "count"
)

In [13]:
analysis0_2.sort_values(by = 'month', ascending = False)

client_id
month week           
12    52      1303718
      51      1166028
      50      1295395
      49      1284341
      48       113101
...               ...
1     4        976478
      3       1269645
      2       1057715
      5        827834
      1       1068966

[64 rows x 1 columns]

## How many products sold per client

In [14]:
analysis0_3 = pd.pivot_table(
data = trans_sample,
values = "quantity",
index = ["client_id"],
aggfunc = "sum"
)

In [15]:
analysis0_3.sort_values(by='quantity', ascending = False)

,quantity
client_id,
1773169,47722799
864026,29856285
243127,28240356
1552980,13352433
1386568,12585785
...,...
330307,3
604174,3
289336,3


## Average overall sales_net

In [16]:
trans_sample['sales_net'].mean()

147.94159215953854

## Product Price Analysis

In [17]:
trans_sample_1 = trans.copy()

In [18]:
#create the price feature
trans_sample_1['price'] = abs(trans_sample_1['sales_net']/trans_sample_1['quantity'])

In [19]:
trans_sample_1.head()

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,price
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732,51.81440
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279,5.46480
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184,17.41744
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200,0.04278
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051,364.45800


In [20]:
trans_sample_1.count() #there is one invoice missing

date_order       63319315
date_invoice     63319314
product_id       63319315
client_id        63319315
sales_net        63319315
quantity         63319315
order_channel    63319315
branch_id        63319315
price            63319315
dtype: int64

In [21]:
trans_sample_1.nunique(axis=0, dropna=True)

date_order           731
date_invoice         900
product_id        607418
client_id         170589
sales_net        2469843
quantity            3444
order_channel          5
branch_id            582
price            1502026
dtype: int64

In [22]:
trans_sample_1['client_id'].value_counts()

417576     105636
1444500     91162
1773169     75883
2207247     64192
1552980     58067
            ...  
2225405         1
805795          1
1604674         1
1061872         1
235746          1
Name: client_id, Length: 170589, dtype: int64

In [23]:
trans_branch_analysis = trans_sample_1.loc[:,['branch_id', 'product_id', 'price']]

In [24]:
trans_branch_analysis.head()

,branch_id,product_id,price
0,2732,2376333,51.81440
1,10279,2520527,5.46480
2,6184,2536756,17.41744
3,4200,3028673,0.04278
4,7051,203377,364.45800


In [25]:
analysis1 = trans_branch_analysis.groupby('product_id').agg({'price': ['mean', 'min', 'max']})#represents the variations of each product prices


In [26]:
analysis1.head()

price                        
                   mean        min          max
product_id                                     
15             0.206535     0.0000     0.950360
23            25.354433    22.8620    27.846867
28          2657.898400  2126.9664  3311.825200
39             4.811600     4.8116     4.811600
42           260.783200   260.7832   260.783200

In [27]:
analysis1['same_price'] = np.where(analysis1['price']['max'] == analysis1['price']['min'],1,0)

In [28]:
product_same_price = analysis1['same_price'].sum()  #352927 out of 607418 products with the same prices throught the branches 

In [29]:
#percentage from the total who have the same price
(product_same_price/ 607418)*100

58.10282210932175

In [30]:
#biggest gap between the prices
analysis1['price_gap'] = analysis1['price']['max'] - analysis1['price']['min']

In [31]:
analysis1.sort_values(by='price_gap', ascending=False).head() #sorted by the biggest gaps

price                         same_price    price_gap
                     mean        min          max                        
product_id                                                               
133186      218715.928600  1993.7412  435438.1160          0  433444.3748
1269937         27.074187     0.0000   91080.0000          0   91080.0000
678008        5247.195955     7.3600   84381.7284          0   84374.3684
3233898        860.071550     0.0000   46118.4776          0   46118.4776
1260130        449.007555     0.0184   34180.8704          0   34180.8520

In [32]:
#can now do the analysis for the top 5 products with a high gap in their prices 133186 - 1269937 - 678008 - 3233898 - 1260130

In [33]:
trans_branch_analysis[trans_branch_analysis['product_id'] == 133186].loc[:,['branch_id','price']] # the difference in price occured in the same branch

,branch_id,price
12635963,4115,435438.1160
32551749,4115,1993.7412


In [34]:
product_1269937 = trans_branch_analysis[trans_branch_analysis['product_id'] == 1269937].loc[:,['branch_id','price']] #present in 5527 transactions 

In [35]:
product_1269937.nunique() #available in 469 branches vs 556 different prices (almost the same number) so we can assume that in each branch we have different prices 

branch_id    469
price        556
dtype: int64

## Branch product price variation analysis

In [36]:
analysis2 = trans_branch_analysis.groupby('branch_id').agg({'price': ['mean', 'min', 'max']})


In [37]:
analysis2.head()

price                 
                mean  min         max
branch_id                            
20         23.959758  0.0   6745.3572
23         10.039500  0.0     13.3860
35         37.324172  0.0  10948.0000
81         26.248803  0.0   4759.1600
83         18.809384  0.0   6437.8380

In [38]:
analysis2['price_gap'] = analysis2['price']['max'] - analysis2['price']['min'] #the smaller it is the less product variations a branch has

In [39]:
analysis2.sort_values(by='price_gap', ascending = False) #branch 4115 seems to be a problematic branch 

price                                price_gap
                 mean         min           max               
branch_id                                                     
4115       436.061670    0.000000  435438.11600  435438.116000
1525        28.929111    0.000000  144092.24000  144092.240000
5489        33.731419    0.000000   96600.00000   96600.000000
2771        33.756022    0.000000   91080.00000   91080.000000
4611        32.738514    0.000000   80128.86280   80128.862800
...               ...         ...           ...            ...
8479         1.665200    0.110400       5.44640       5.336000
7679         0.476184    0.120944       4.67222       4.551276
6254         0.512133    0.377200       0.58880       0.211600
2925        13.588400   13.588400      13.58840       0.000000
9125       426.571800  426.571800     426.57180       0.000000

[582 rows x 4 columns]